In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


from teste import diretorio_teste
project_path=diretorio_teste()

In [2]:
import os
import geopandas as gpd
import pandas as pd
from logging import (
    Logger,
    getLogger,
)
Logger=getLogger()

from notebooks.jupyter import utils
from utils import (
    save_parquet_excel,
    get_data_diretorio,
    check_crs
)

In [3]:
data_path = get_data_diretorio()

# Parques Urbanos e Lineares

# Dependências

Este notebook é dependente dos assets resultante dos notebooks '../../area_verde_publica/download_data/malha_parques_municipais.ipynb'

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

In [5]:
gdf_parq_mun = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_municipais.parquet'
    )
)

# Conferir e Filtrar Parques Lineares

In [6]:
gdf_parq_mun.sample(2)

,cd_parq_mun,nm_parque,tx_tipo_ca,tx_tipo_si,cd_tipo_ca,cd_tipo_si,geometry
165,181,Oyeno,Parque Urbano,Proposto,1,1,"MULTIPOLYGON (((339631.572 7399570.261, 339554..."
143,7,Aterro Itatinga,Parque Urbano,Proposto,1,1,"POLYGON ((333229.538 7378685.21, 333246.186 73..."


In [7]:
gdf_parq_mun['tx_tipo_ca']=gdf_parq_mun['tx_tipo_ca'].str.lower()

In [8]:
uniques=gdf_parq_mun['tx_tipo_ca'].unique()
#.str.contains('inear', case=False)
uniques

array(['parque urbano', 'parque orla', 'parque de conservação',
       'parque natural', 'parque linear'], dtype=object)

In [9]:
for u in uniques:
    if 'linear' in str(u):
        name_linear = u
    if 'urb' in str(u):
        name_urb=u
print (f'{name_linear} | {name_urb}')

parque linear | parque urbano


def get_cd(gdf, name_1, name_2:str='', name_3:str='')->pd.Series:
    num = 1
    name = name_1
    for name !=''
        cd =(
            gdf.loc[gdf['tx_tipo_ca']==name, 'cd_tipo_ca']
            .unique()
        )
    
    


In [10]:
cd_linear=(
    gdf_parq_mun
    .loc[gdf_parq_mun['tx_tipo_ca']==name_linear, 'cd_tipo_ca']
    .unique()
)

print(cd_linear)

if int(cd_linear.shape[0])==1:
    cd_linear = int(cd_linear[0])
else:
    Logger.error(
        f'''Erro: o cd_tipo_ca ({
            cd_linear
        }) não corresponde completamente ao txt_tipo_ca({
            name_linear
        })'''
    )

cd_linear

[5]


5

In [11]:
cd_urb = (
    gdf_parq_mun
    .loc[gdf_parq_mun['tx_tipo_ca']==name_urb, 'cd_tipo_ca']
    .unique()
)

if int(cd_urb.shape[0])==1:
    cd_urb = int(cd_urb[0])
else:
    Logger.error(
        f'''Erro: o cd_tipo_ca ({
            cd_urb
        }) não corresponde completamente ao txt_tipo_ca({
            name_urb
        })'''
    )

cd_urb

1

Aqui descobrimos que o cd dos parques lineares (presente em cd_tipo_ca) é 5 e dos parques urbanos é 1. Vamos filtrar pelo cd, pois ele é mais confiável que astr 

In [12]:
parques_line_urb=pd.concat([
    (
        gdf_parq_mun
        .loc[gdf_parq_mun['cd_tipo_ca']== cd_linear]
    ),
    (
        gdf_parq_mun
        .loc[gdf_parq_mun['cd_tipo_ca']==cd_urb]
    )
])

In [13]:
parques_line_urb.shape

(213, 7)

In [14]:
parques_line_urb.sample(3)

,cd_parq_mun,nm_parque,tx_tipo_ca,tx_tipo_si,cd_tipo_ca,cd_tipo_si,geometry
143,7,Aterro Itatinga,parque urbano,Proposto,1,1,"POLYGON ((333229.538 7378685.21, 333246.186 73..."
191,12,Ipiranguinha,parque linear,Existente,5,2,"MULTIPOLYGON (((343905.91 7391998.678, 343905...."
99,164,Jardim das Perdizes,parque urbano,Existente,1,2,"MULTIPOLYGON (((328170.974 7398112.523, 328174..."


# Conferir CRS

In [15]:
parques_line_urb= check_crs(parques_line_urb)

In [17]:
parques_line_urb.explore()

# Salvar GDF

In [16]:
save_parquet_excel(
    parques_line_urb,
    fname='parques_urbanos_e_lineares',
    data_path = data_path,
    data_subpath = 'assets'
)